In [5]:
import json, random,os
import matplotlib.pyplot as plt
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from tqdm.notebook import tqdm
from langchain_community.retrievers import BM25Retriever

DATA_DIR = "/mnt/data/factcheck/averimatec"
SPLIT = "val"
EMBEDDING_MODEL, EMBEDDING_NAME = "mixedbread-ai/mxbai-embed-large-v1", "mxbai"
CROP_SIZE, CROP_SIZE_STR = 2000, "2k"
TOKENS_PER_CHAR = 0.25
EMBEDDING_INPUT_SIZE = 512 # Mxbai-Embedding max input size

embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL, model_kwargs={"device": "cuda:0"})
random.seed(111)

with open(f"{DATA_DIR}/{SPLIT}.json") as f:
    datapoints = json.load(f)

In [6]:
datapoints[2]

{'article': 'https://web.archive.org/web/20230629073625/https://leadstories.com/hoax-alert/2023/06/fact-check-photos-do-not-prove-hobby-lobby-is-selling-satanic-statues.html',
 'date': '2023-06-13',
 'label': 'Refuted',
 'location': 'US',
 'questions': [{'input_images': [],
   'answers': [{'answer_type': 'Extractive',
     'answer_text': 'Hobby Lobby Stores, Inc., formerly Hobby Lobby Creative Centers, is an American retail company. It owns a chain of arts and crafts stores with a volume of over $5 billion in 2018.',
     'source_url': 'https://en.wikipedia.org/wiki/Hobby_Lobby',
     'source_medium': 'Web text'}],
   'question': 'What is Hobby Lobby?',
   'answer_method': 'Text-search',
   'question_type': ['Text-related']},
  {'input_images': ['6787dee1e2f02e5f4981b14e#INPUT#1#0.jpg',
    '6787dee1e2f02e5f4981b14e#INPUT#1#1.jpg',
    '6787dee1e2f02e5f4981b14e#INPUT#1#2.jpg',
    '6787dee1e2f02e5f4981b14e#INPUT#1#3.jpg'],
   'answers': [{'answer_type': 'Abstractive',
     'answer_text

In [7]:
for CLAIM_ID in tqdm(range(0,len(datapoints))):# Naive version with \n concatenated url2texts:   
    # skip if f"{DATA_DIR}/data_store/vecstore/{SPLIT}/4k/{CLAIM_ID}" exists
    if os.path.exists(f"{DATA_DIR}/vector_store/{SPLIT}/{CROP_SIZE_STR}_{EMBEDDING_NAME}/{CLAIM_ID}"):
        continue

    os.makedirs(f"{DATA_DIR}/vector_store/{SPLIT}/{CROP_SIZE_STR}_{EMBEDDING_NAME}/{CLAIM_ID}")

    datapoint = datapoints[CLAIM_ID]
    claim = datapoint["claim_text"]
    # display(Markdown("### 🗯️ " + claim + " [" + datapoint["label"] + "]"))
    docstore = []
    # /mnt/data/factcheck/averimatec/knowledge_store/val/text_related/text_related_store_text_val
    for line in open(f"{DATA_DIR}/knowledge_store/{SPLIT}/text_related/text_related_store_text_{SPLIT}/{CLAIM_ID}.json"):
        docstore.append(json.loads(line))

    documents = [
        Document(
            page_content=" ".join(doc["url2text"]),
            metadata={
                "url": doc["url"],
                # "sentences": doc["url2text"]
            },
        )
        for doc in docstore
    ]

    chunks = []
    for doc in docstore:
        if not doc["url2text"]:
            continue
        buffer = ""
        for i, sentence in enumerate(doc["url2text"]):
            if (
                i == len(doc["url2text"]) - 1
                or len(buffer) + len(sentence) >= EMBEDDING_INPUT_SIZE / TOKENS_PER_CHAR
            ):
                context_before = ""
                if chunks and chunks[-1].metadata["url"] == doc["url"]:
                    chunks[-1].metadata["context_after"] = buffer
                    context_before = chunks[-1].page_content
                chunks.append(
                    Document(
                        page_content=buffer,
                        metadata={"url": doc["url"], "context_before": context_before, "context_after": ""},
                    )
                )

                buffer = ""
            buffer += sentence + " "
    # chunk the documents into smaller pieces
    chid = random.randint(0, len(chunks))

    # display(Markdown(chunks[chid].metadata["context_before"]))
    # display(Markdown(chunks[chid].page_content))
    # display(Markdown(chunks[chid].metadata["context_after"]))

    # print(chunks[chid].page_content)
    retriever = BM25Retriever.from_documents(
        chunks, k=CROP_SIZE
    )
    chunks_pruned = retriever.invoke(claim)
    db = FAISS.from_documents(chunks_pruned, embeddings)
    db.save_local(f"{DATA_DIR}/vector_store/{SPLIT}/{CROP_SIZE_STR}_{EMBEDDING_NAME}/{CLAIM_ID}")

  0%|          | 0/152 [00:00<?, ?it/s]

In [ ]:
# upgrade sentence_transformers and transformers to the latest version
%pip install -U sentence-transformers transformers

In [ ]:
# check which device embeddings use
import torch
print(torch.cuda.is_available(), torch.cuda.current_device(), torch.cuda.get_device_name(0))
print(torch.__version__)
embeddings = HuggingFaceEmbeddings(model_name="google/embeddinggemma-300m")
#make sure the embeddings are on cuda
print(embeddings.client.device)